# Analysis of Chris Hughes' "Fair Shot" Plan

Analysis of cash transfer plan proposed by Chris Hughes in his 2018 book, [*FAIR SHOT: Rethinking Inequality and How We Earn*](http://www.fairshotbook.com/).

Per [Felix Salmon's review](https://www.nytimes.com/2018/02/25/books/review/chris-hughes-fair-shot.html) (summarized in my [Twitter thread](https://twitter.com/MaxGhenis/status/968216600670171136)), the elements of the plan are:

1) \$6k/year paid monthly to each adult in a household with income between \$6k-\$50k in previous year, or who earned <\$50k but (a) had cared for a dependent under age six or over age 70, or (b) were enrolled in an accredited college.

2) Households earning <\$6k in past year and who don't meet dependent or college requirements get their previous year's earnings matched 100%.

3) Funded by increasing to 50% the marginal rate on income and capital gains for households earning above \$250k.

Tax-Calculator CPS data ([documentation](http://open-source-economics.github.io/Tax-Calculator/)) lacks certain data elements necessary to fully implement this, so we make simplifying assumptions.

* **No ages of child dependents.** Instead flag tax units with at least one person under age 18 (`nu18 > 0`).
* **No ages of elderly parent dependents.** Instead flag all tax units that claimed an elderly parent as a dependent (`elderly_dependent`, which has [no age threshold](https://turbotax.intuit.com/tax-tips/family/steps-to-claiming-an-elderly-parent-as-a-dependent/L34jePeT9)).
* **No college enrollment.** Instead flag tax units with nonzero education tax credits (`c07230 > 0`).
* **No knowledge of the number of adults working.** Instead give for each adult in the tax unit.

To simplify, this also gives money to the tax units in the same year. It's unclear whether taxdata models longitudinal variance of tax units' incomes and other situations.

Potential child care fields include:

* `e32800`: "Child/dependent-care expenses for qualifying persons from Form 2441"
* `f2441`: "number of child/dependent-care qualifying persons"
* `c07180`: "Credit for child and dependent care expenses from Form 2441"

*Data: CPS  |  Tax year: 2018  |  Type: Static  |  Author: Max Ghenis  |  Date run: 2018-03-01*

## Setup

### Imports

In [2]:
import taxcalc as tc
import pandas as pd
import numpy as np
import copy
from bokeh.io import show, output_notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import urllib as url_lib  # On Python 3.6 use "import urllib.request as url_lib".

In [3]:
tc.__version__

'0.16.1'

In [4]:
sns.set_style('white')
DPI = 75
mpl.rc('savefig', dpi=DPI)
mpl.rcParams['figure.dpi']= DPI
mpl.rcParams['figure.figsize'] = 6.4, 4.8  # Default.

In [5]:
mpl.rcParams['font.sans-serif'] = 'Roboto'
mpl.rcParams['font.family'] = 'sans-serif'

# Set title text color to dark gray (https://material.io/color) not black.
TITLE_COLOR = '#212121'
mpl.rcParams['text.color'] = TITLE_COLOR

# Axis titles and tick marks are medium gray.
AXIS_COLOR = '#757575'
mpl.rcParams['axes.labelcolor'] = AXIS_COLOR
mpl.rcParams['xtick.color'] = AXIS_COLOR
mpl.rcParams['ytick.color'] = AXIS_COLOR

# Use Seaborn's default color palette.
# https://stackoverflow.com/q/48958426/1840471 for reproducibility.
sns.set_palette(sns.color_palette())

In [57]:
# Show one decimal in tables.
pd.set_option('precision', 2)

### Utility functions

In [53]:
def weighted_sum(df, col):
    return (df[col] * df['s006']).sum()

In [7]:
def gini(x, weights=None):
    if weights is None:
        weights = np.ones_like(x)
    # Calculate mean absolute deviation in two steps, for weights.
    count = np.multiply.outer(weights, weights)
    mad = np.abs(np.subtract.outer(x, x) * count).sum() / count.sum()
    # Gini equals half the relative mean absolute deviation.
    rmad = mad / np.average(x, weights=weights)
    return 0.5 * rmad

## Model

Create `DataFrame`s for the baseline and reform scenarios. Only the tax increase is modeled as part of the taxcalc reform, since the particular form of the cash transfer isn't available. Instead it's added separately.

### Specify reform

For all tax brackets with thresholds exceeding 250k, set to 250k. Set to 50% all brackets.

In [26]:
reform = {
    2018: {
        '_II_brk4': [[157500.0, 250000.0, 157500.0, 157500.0, 250000.0]],
        '_II_brk5': [[200000.0, 250000.0, 200000.0, 200000.0, 250000.0]],
        '_II_brk6': [[250000.0, 250000.0, 250000.0, 250000.0, 250000.0]],
        '_II_rt7': [0.5]
    }
}

### Run Tax-Calculator

In [11]:
recs = tc.Records.cps_constructor()

0% the marginal rate on income and capital gains for households earning above $250k.

Tax-Calculator CPS data (documentation) lacks certain data elements necessary to fully implement this, so we make simplifying assumptions.

No ages of child dependents. Instead flag tax units with at least one person under age 18 (nu18 > 0).
No ages of elderly parent dependents. Instead flag all tax units that claimed an elderly parent as a dependent (elderly_dependent, which has no age threshold).
No college enrollment. Instead flag tax units with nonzero education tax credits (c07230 > 0).
No knowledge of the number of adults working. Instead give for each adult in the tax unit.
To simplify, this also gives money to the tax units in the same year. It's unclear whether taxdata models longitudinal variance of tax units' incomes and other situations.

Potential child care fields include:

e32800: "Child/dependent-care expenses for qualifying persons from Form 2441"
f2441: "number of child/dependent-care qualifying persons"
c07180:

In [65]:
base_calc = tc.Calculator(records=recs, policy=tc.Policy())
base_calc.advance_to_year(2018)
base_calc.calc_all()

base_df = base_calc.dataframe(['s006', 'nu18', 'XTOT', 'elderly_dependent', 
                               'c07230', 'expanded_income', 'aftertax_income'])

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [37]:
pol = tc.Policy()
pol.implement_reform(reform)
reform_calc = tc.Calculator(records=recs, policy=pol)
reform_calc.advance_to_year(2018)
reform_calc.calc_all()

reform_df = reform_calc.dataframe(['aftertax_income'])

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


### Merge base and reform

In [66]:
base_df.rename(columns={'aftertax_income':'aftertax_income_base'}, inplace=True)
reform_df.rename(columns={'aftertax_income':'aftertax_income_reform_preubi'}, 
                 inplace=True)

In [67]:
df = pd.merge(base_df, reform_df, left_index=True, right_index=True)

### Add FairShot cash amount

Pseudocode:
```
if expanded_income > 50000 then 0
elif elderly_dependent > 0 or nu18 > 0 or expanded_income > (6000 * (XTOT - nu18)) 
  then 6000 * (XTOT - nu18)
else expanded_income
```

1. Flag eligibility using expanded income.
2. Calculate maximum possible amount as `6000 * (XTOT - nu18)`.
3. Calculate eligibility for full amount based on dependents and college.
4. Calculate amount as ((1) if (2)) otherwise just expanded income.

In [69]:
df.dtypes

s006                             float64
nu18                             float64
XTOT                             float64
elderly_dependent                float64
c07230                           float64
expanded_income                  float64
aftertax_income_base             float64
aftertax_income_reform_preubi    float64
dtype: object

In [73]:
np.argmax(np.array([2, 2]))

0

In [78]:
np.where([True, False], [1, 3], [2, 4])

array([1, 4])

In [85]:
df['fairshot_eligible'] = df['expanded_income'] < 50000
df['fairshot_max'] = 6000 * (df['XTOT'] - df['nu18'])
df['fairshot_max_eligible'] = (
    (df['elderly_dependent'] > 0) | 
    (df['nu18'] > 0) |
    (df['c07230'] > 0) |
    (df['expanded_income'] >= df['fairshot_max']))
df['fairshot'] = np.where(df['fairshot_eligible'], 
                          np.where(df['fairshot_max_eligible'],
                                   df['fairshot_max'],
                                   df['expanded_income']),
                          0)

In [87]:
df.groupby('c07230')['s006'].sum()

c07230
0.0    1.70e+08
Name: s006, dtype: float64

In [86]:
df[df['expanded_income'] < 6000].sample(5)

,s006,nu18,XTOT,elderly_dependent,c07230,expanded_income,aftertax_income_base,aftertax_income_reform_preubi,fairshot_eligible,fairshot_max,fairshot_max_eligible,fairshot
440233,472.20,0.0,1.0,0.0,0.0,1772.41,1520.50,1520.50,True,6000.0,False,1772.41
158456,1281.16,0.0,1.0,0.0,0.0,4100.04,4100.04,4100.04,True,6000.0,False,4100.04
271809,1146.34,0.0,2.0,0.0,0.0,3248.73,3017.86,3017.86,True,12000.0,False,3248.73
89353,532.08,0.0,1.0,0.0,0.0,4447.20,4393.49,4393.49,True,6000.0,False,4447.20
219284,1314.24,0.0,1.0,0.0,0.0,4534.26,4534.26,4534.26,True,6000.0,False,4534.26


## Analysis

### Calculate revenue from tax increases

In [64]:
'Tax increases would raise ${0:0.1f}B in revenue.'.format(
    (weighted_sum(df, 'aftertax_income_base') -
     weighted_sum(df, 'aftertax_income_reform_preubi')) / 1e9)

'Tax increases would raise $86.7B in revenue.'